In [2]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.TwoDoF as TwoDof
import FwtModels as fm

from matplotlib.lines import Line2D

me.mechanics_printing()

In [3]:
half = sym.Rational(1,2)

p = TwoDof.base_params()

In [35]:
#define refernce frames
wing_root_frame = mf.HomogenousTransform().Translate(0,0,p.q[0]).R_x(-p.q[1])
wing_aero_frame = wing_root_frame.Translate(0,p.y_0,0)
wing_com_frame = wing_root_frame.Translate(0,p.l_com,0)

#Create Elemnts
M_fwt = ele.MassMatrix(p.m,I_xx = p.I_xx)
M_innerWing = ele.RigidElement.PointMass(wing_root_frame,p.m_w,True)

fwt_ele = ele.RigidElement(wing_com_frame,M_fwt,True)
spring_ele = ele.Spring(p.q[0],p.k_w)
spring_ele_fwt = ele.Spring(p.q[1],p.k_fwt)


# Create AeroForces
wing_AeroForces = ef.AeroForce.PerUnitSpan(p,wing_aero_frame,p.a_0,
                               alphadot = p.alphadot_1,
                               M_thetadot = p.M_thetadot,
                               e = 0,
                               w_g = 0,
                               rootAlpha = p.alpha_1,
                               alpha_zero = 0,
                               c = p.c,
                            c_d_max = 0,stall_angle=0,linear=True)#.integrate((p.y_0,0,p.s))

# Setup AoA of FWT
#fwt_aoa = fm.GetAoA(p.alpha_r,0,p.Lambda,p.q[1])

## Sub in Aero Forces
#wing_AeroForces = wing_AeroForces.subs({p.alpha_1:fwt_aoa,p.alphadot_1:fwt_aoa.diff(time)})

forces = [wing_AeroForces]
forces = []
segments = 1
for i in range(segments):
    seg_width = p.s/segments
    yi = seg_width/2 + i*seg_width
    forces.append(wing_AeroForces.subs({p.y_0:yi})*seg_width)
Q = sym.Matrix([0]*p.qs)
for f in forces:
    Q += f.Q()
c_forces = ef.ExternalForce(Q)

In [36]:
sm = mf.SymbolicModel.FromElementsAndForces(p,[M_innerWing,fwt_ele,spring_ele,spring_ele_fwt],c_forces)

In [37]:
sm.to_file('2Dof_Model.py')

In [38]:
sm.M

⎡    m + m_w       -l_com⋅m⋅cos(q₁)⎤
⎢                                  ⎥
⎢                              2   ⎥
⎣-l_com⋅m⋅cos(q₁)   Iₓₓ + l_com ⋅m ⎦

In [39]:
sm.f

⎡                                         2⎤
⎢g⋅m + g⋅m_w + k_w⋅q₀ + l_com⋅m⋅sin(q₁)⋅q₁̇ ⎥
⎢                                          ⎥
⎣      -g⋅l_com⋅m⋅cos(q₁) + k_fwt⋅q₁       ⎦

In [40]:
sm.ExtForces.Q()

⎡            ⎛       s⋅q₁̇             ⎞        ⎤
⎢            ⎜     - ──── + cos(q₁)⋅q₀̇⎟        ⎥
⎢ 2          ⎜        2               ⎟        ⎥
⎢V ⋅a₀⋅c⋅ρ⋅s⋅⎜α₁ - ───────────────────⎟⋅cos(q₁)⎥
⎢            ⎝              V         ⎠        ⎥
⎢──────────────────────────────────────────────⎥
⎢                      2                       ⎥
⎢                                              ⎥
⎢                ⎛       s⋅q₁̇             ⎞    ⎥
⎢                ⎜     - ──── + cos(q₁)⋅q₀̇⎟    ⎥
⎢    2         2 ⎜        2               ⎟    ⎥
⎢  -V ⋅a₀⋅c⋅ρ⋅s ⋅⎜α₁ - ───────────────────⎟    ⎥
⎢                ⎝              V         ⎠    ⎥
⎢  ─────────────────────────────────────────   ⎥
⎣                      4                       ⎦